In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [11]:
import os
import tensorflow as tf
import numpy as np

# Set the seed for random operations. 
# This let our experiments to be reproducible. 
SEED = 1234
tf.random.set_seed(SEED)  

# Get current working directory
cwd = os.getcwd()

# Set GPU memory growth 
# Allows to only as much GPU memory as needed
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

# Set up the image generator with some random augmentation

In [12]:
# ImageDataGenerator

from tensorflow.keras.preprocessing.image import ImageDataGenerator

apply_data_augmentation = False

# Create training ImageDataGenerator object
train_data_gen = ImageDataGenerator(rotation_range=10,
                                        width_shift_range=10,
                                        height_shift_range=10,
                                        zoom_range=0.3,
                                        horizontal_flip=True,
                                        vertical_flip=True,
                                        fill_mode='constant',
                                        cval=0,
                                        rescale=1./255,
                                        validation_split=0.2 # NOTE THIS VERY WELL
                                   )

In [13]:
# Defining training directory
from os.path import dirname
parent_cwd = dirname(cwd)
dataset_dir = os.path.join(parent_cwd, "data")
dataset_dir = os.path.join(dataset_dir, "Classification_Dataset")
dataset_dir = os.path.join(dataset_dir, "old_train","training")

In [14]:
# Create generators to read images from dataset directory
# -------------------------------------------------------
# Batch size
bs = 8

# img shape
img_h = 256
img_w = 256

num_classes=20

decide_class_indices = True
if decide_class_indices:
    classes = ['school-bus',           #0
                'laptop',              #1
                't-shirt',             #2 
                'grand-piano',         #3
                'waterfall',           #4
                'galaxy',              #5 
                'mountain-bike',       #6
                'sword',               #7
                'wine-bottle',         #8
                'owl',                 #9
                'fireworks',           #10
                'calculator',          #11
                'sheet-music',         #12
                'lightbulb',           #13
                'bear',                #14
                'computer-monitor',    #15
                'airplanes',           #16
                'skyscraper',          #17
                'lightning',           #18
                'kangaroo']          #19


# Training
train_gen = train_data_gen.flow_from_directory(dataset_dir,
                                               batch_size=bs,
                                               classes=classes,
                                               class_mode='categorical',
                                               shuffle=True,
                                               seed=SEED,
                                               subset="training")  # targets are directly converted into one-hot vectors

valid_gen = train_data_gen.flow_from_directory(dataset_dir,
                                               batch_size=bs,
                                               classes=classes,
                                               class_mode='categorical',
                                               shuffle=True,
                                               seed=SEED,
                                               subset="validation")  # targets are directly converted into one-hot vectors


Found 1247 images belonging to 20 classes.
Found 307 images belonging to 20 classes.


In [15]:
train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))

valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen, 
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))
train_dataset = train_dataset.repeat()
valid_dataset = valid_dataset.repeat()

In [16]:
# Let's test data augmentation
# ----------------------------

iterator = iter(valid_dataset)

owl_count = 0

for _ in range(1):
    augmented_img, target = next(iterator)
    augmented_img = augmented_img[0]   # First element
    augmented_img = augmented_img * 255  # denormalize
    
    print(target)

tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(8, 20), dtype=float32)


# Let's build our first CNN

In [17]:
# Keras Model subclassing 
# Create convolutional block
class ConvBlock(tf.keras.Model):
    def __init__(self, num_filters):
        super(ConvBlock, self).__init__()
        self.conv2d = tf.keras.layers.Conv2D(filters=num_filters,
                                             kernel_size=(3, 3),
                                             strides=(1, 1), 
                                             padding='same')
        self.activation = tf.keras.layers.ReLU()  # we can specify the activation function directly in Conv2D
        self.pooling = tf.keras.layers.MaxPool2D(pool_size=(2, 2))
        
    def call(self, inputs):
        x = self.conv2d(inputs)
        x = self.activation(x)
        x = self.pooling(x)
        return x

In [18]:
class FirstCNNClassifier(tf.keras.Model):
    def __init__(self, depth, start_f, num_classes, dense_units=512, activation_dense="relu",
                number_dense_layer=1, dropout=True):
        super(FirstCNNClassifier, self).__init__()
        
        self.feature_extractor = tf.keras.Sequential()
    
        for i in range(depth):
            self.feature_extractor.add(ConvBlock(num_filters=start_f))
            start_f *= 2
            
        self.flatten = tf.keras.layers.Flatten()
        self.classifier = tf.keras.Sequential()
        
        for i in range(number_dense_layer):
            self.classifier.add(tf.keras.layers.Dense(units=dense_units, activation=activation_dense))
        
        if dropout:
            self.classifier.add(tf.keras.layers.Dropout(rate=0.5, seed=SEED))
        
        self.classifier.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))
        
    def call(self, inputs):
        x = self.feature_extractor(inputs)
        x = self.flatten(x)
        x = self.classifier(x)
        return x

In [19]:
# Create Model
# ------------

depth = 5
start_f = 8
num_classes = 20
    
# Create Model instance
model = FirstCNNClassifier(depth=depth,
                      start_f=start_f,
                      num_classes=num_classes,
                      dropout=True)
# Build Model (Required)
model.build(input_shape=(None, img_h, img_w, 3))

In [20]:
model.summary()

Model: "first_cnn_classifier"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      multiple                  98384     
_________________________________________________________________
flatten (Flatten)            multiple                  0         
_________________________________________________________________
sequential_1 (Sequential)    multiple                  4205076   
Total params: 4,303,460
Trainable params: 4,303,460
Non-trainable params: 0
_________________________________________________________________


In [21]:
model.feature_extractor.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_block (ConvBlock)       multiple                  224       
_________________________________________________________________
conv_block_1 (ConvBlock)     multiple                  1168      
_________________________________________________________________
conv_block_2 (ConvBlock)     multiple                  4640      
_________________________________________________________________
conv_block_3 (ConvBlock)     multiple                  18496     
_________________________________________________________________
conv_block_4 (ConvBlock)     multiple                  73856     
Total params: 98,384
Trainable params: 98,384
Non-trainable params: 0
_________________________________________________________________


# Prepare model for training

In [22]:
# Optimization params
# -------------------

# Loss
loss = tf.keras.losses.CategoricalCrossentropy()

# learning rate
lr = 1e-3
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
# -------------------

# Validation metrics
# ------------------

metrics = ['accuracy']
# ------------------

# Compile Model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

# Traing with callbacks

In [23]:
import os
from datetime import datetime

# from tensorflow.compat.v1 import ConfigProto
# from tensorflow.compat.v1 import InteractiveSession

# config = ConfigProto()
# config.gpu_options.allow_growth = True
# session = InteractiveSession(config=config)

cwd = os.getcwd()

exps_dir = os.path.join(cwd, 'classification_experiments')
if not os.path.exists(exps_dir):
    os.makedirs(exps_dir)

now = datetime.now().strftime('%b%d_%H-%M-%S')

model_name = 'CNN'

exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)
    
callbacks = []

# Model checkpoint
# ----------------
ckpt_dir = os.path.join(exp_dir, 'ckpts')
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp_{epoch:02d}.ckpt'), 
                                                   save_weights_only=True)  # False to save the model directly
callbacks.append(ckpt_callback)

# Visualize Learning on Tensorboard
# ---------------------------------
tb_dir = os.path.join(exp_dir, 'tb_logs')
if not os.path.exists(tb_dir):
    os.makedirs(tb_dir)
    
# By default shows losses and metrics for both training and validation
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir,
                                             profile_batch=0,
                                             histogram_freq=1)  # if 1 shows weights histograms
callbacks.append(tb_callback)

# Early Stopping
# --------------
early_stop = False
if early_stop:
    es_callback = tf.keras.callback.EarlyStopping(monitor='val_loss', patience=10)
    callbacks.append(es_callback)


model.fit(x=train_dataset,
          epochs=100,  #### set repeat in training dataset
          steps_per_epoch=len(train_gen),
          validation_data=valid_dataset,
          validation_steps=len(valid_gen), 
          callbacks=callbacks)

# How to visualize Tensorboard

# 1. tensorboard --logdir EXPERIMENTS_DIR --port PORT     <- from terminal
# 2. localhost:PORT   <- in your browser

Train for 156 steps, validate for 39 steps
Epoch 1/100
 60/156 [==========>...................] - ETA: 55s - loss: 3.0144 - accuracy: 0.0720     

Exception ignored in: <bound method Comm.__del__ of <ipykernel.comm.comm.Comm object at 0x0000022660209828>>
Traceback (most recent call last):
  File "C:\Users\Riccardo\Anaconda3\envs\anndl\lib\site-packages\ipykernel\comm\comm.py", line 76, in __del__
    self.close()
  File "C:\Users\Riccardo\Anaconda3\envs\anndl\lib\site-packages\ipykernel\comm\comm.py", line 116, in close
    self.kernel.comm_manager.unregister_comm(self)
  File "C:\Users\Riccardo\Anaconda3\envs\anndl\lib\site-packages\ipykernel\comm\manager.py", line 56, in unregister_comm
    comm = self.comms.pop(comm.comm_id)
KeyError: ('019895cb751f487094d4f7710013cbdb',)
Exception ignored in: <bound method Comm.__del__ of <ipykernel.comm.comm.Comm object at 0x0000022661E88F60>>
Traceback (most recent call last):
  File "C:\Users\Riccardo\Anaconda3\envs\anndl\lib\site-packages\ipykernel\comm\comm.py", line 76, in __del__
    self.close()
  File "C:\Users\Riccardo\Anaconda3\envs\anndl\lib\site-packages\ipykernel\comm\comm.py",

KeyboardInterrupt: 

# Model restoration and calculating accuracy without augmenting the validation set

In [ ]:
# ImageDataGenerator

from tensorflow.keras.preprocessing.image import ImageDataGenerator

apply_data_augmentation = False

# Create training ImageDataGenerator object
train_data_gen = ImageDataGenerator(rotation_range=10,
                                        width_shift_range=10,
                                        height_shift_range=10,
                                        zoom_range=0.3,
                                        horizontal_flip=True,
                                        vertical_flip=True,
                                        fill_mode='constant',
                                        cval=0,
                                        rescale=1./255,
                                        validation_split=0.2 # NOTE THIS VERY WELL
                                   )

In [ ]:
# Defining training directory
from os.path import dirname
parent_cwd = dirname(cwd)
dataset_dir = os.path.join(parent_cwd, "data")
dataset_dir = os.path.join(dataset_dir, "Classification_Dataset")
dataset_dir = os.path.join(dataset_dir, "training")

In [ ]:
# Create generators to read images from dataset directory
# -------------------------------------------------------
# Batch size
bs = 8

# img shape
img_h = 256
img_w = 256

num_classes=20

decide_class_indices = True
if decide_class_indices:
    classes = [
            'owl',  # 1
            'galaxy',  # 2
            'lightning',  # 3
            'wine-bottle',  # ...
            't-shirt',
            'waterfall',
            'sword',
            'school-bus',
            'calculator',
            'sheet-music',
            'airplanes',
            'lightbulb',
            'skyscraper',
            'mountain-bike',
            'fireworks',
            'computer-monitor',
            'bear',
            'grand-piano',
            'kangaroo',
            'laptop'
        ]
# Training
train_gen = train_data_gen.flow_from_directory(dataset_dir,
                                               batch_size=bs,
                                               classes=classes,
                                               class_mode='categorical',
                                               shuffle=True,
                                               seed=SEED,
                                               subset="training")  # targets are directly converted into one-hot vectors

valid_gen = train_data_gen.flow_from_directory(dataset_dir,
                                               batch_size=bs,
                                               classes=classes,
                                               class_mode='categorical',
                                               shuffle=True,
                                               seed=SEED,
                                               subset="validation")  # targets are directly converted into one-hot vectors


In [ ]:
train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))

valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen, 
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))
train_dataset = train_dataset.repeat()
valid_dataset = valid_dataset.repeat()

In [ ]:
# Just for exercise try to restore a model after training it
# !! Use this just when restoring model.. 
# ---------------------------------------
restore_model = True
if restore_model:
    depth = 5
    start_f = 8
    num_classes = 20
    
    # Create Model instance
    model = FirstCNNClassifier(depth=depth,
                      start_f=start_f,
                      num_classes=num_classes,
                      dropout=True)
    
    model.build(input_shape=(None, img_h, img_w, 3))
    loss = tf.keras.losses.CategoricalCrossentropy() 
    metrics = ['accuracy']
    model.compile(loss=loss, metrics=metrics)
    
    weights_dir = os.path.join(cwd, 'classification_experiments', 'CNN_Nov16_15-24-15', 'ckpts', 'cp_98.ckpt')
    
    model.load_weights(weights_dir)  # use this if you want to restore saved model


In [ ]:
# model.load_weights('/path/to/checkpoint')  # use this if you want to restore saved model
eval_out = model.evaluate(x=valid_dataset, steps = 307, verbose=0)

eval_out

# New model

In [ ]:
# ImageDataGenerator

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create training ImageDataGenerator object
train_data_gen = ImageDataGenerator(rotation_range=10,
                                        width_shift_range=10,
                                        height_shift_range=10,
                                        zoom_range=0.3,
                                        horizontal_flip=True,
                                        vertical_flip=True,
                                        fill_mode='constant',
                                        cval=0,
                                        rescale=1./255)                                   
valid_data_gen = ImageDataGenerator(rescale=1./255)


In [ ]:
# Defining training directory
from os.path import dirname
parent_cwd = dirname(cwd)
dataset_dir = os.path.join(parent_cwd, "data")
dataset_dir = os.path.join(dataset_dir, "Classification_Dataset")
train_dir = os.path.join(dataset_dir, "training")
valid_dir = os.path.join(dataset_dir, "validation")
# Create generators to read images from dataset directory
# -------------------------------------------------------
# Batch size
bs = 32

# img shape
img_h = 256
img_w = 256

num_classes=20

decide_class_indices = True
if decide_class_indices:
    classes = [
            'owl',  # 1
            'galaxy',  # 2
            'lightning',  # 3
            'wine-bottle',  # ...
            't-shirt',
            'waterfall',
            'sword',
            'school-bus',
            'calculator',
            'sheet-music',
            'airplanes',
            'lightbulb',
            'skyscraper',
            'mountain-bike',
            'fireworks',
            'computer-monitor',
            'bear',
            'grand-piano',
            'kangaroo',
            'laptop'
        ]

# Training
train_gen = train_data_gen.flow_from_directory(train_dir,
                                               batch_size=bs,
                                               classes=classes,
                                               class_mode='categorical',
                                               shuffle=True,
                                               seed=SEED)  # targets are directly converted into one-hot vectors

valid_gen = valid_data_gen.flow_from_directory(valid_dir,
                                               batch_size=bs,
                                               classes=classes,
                                               class_mode='categorical',
                                               shuffle=True,
                                               seed=SEED)  # targets are directly converted into one-hot vectors


In [ ]:
# Keras Model subclassing 
# Create convolutional block
class ConvBlock(tf.keras.Model):
    def __init__(self, num_filters):
        super(ConvBlock, self).__init__()
        self.conv2d = tf.keras.layers.Conv2D(filters=num_filters,
                                             kernel_size=(3, 3),
                                             strides=(1, 1), 
                                             padding='same')
        self.activation = tf.keras.layers.ReLU()  # we can specify the activation function directly in Conv2D
        self.pooling = tf.keras.layers.MaxPool2D(pool_size=(2, 2))
        
    def call(self, inputs):
        x = self.conv2d(inputs)
        x = self.activation(x)
        x = self.pooling(x)
        return x

In [ ]:
class FirstCNNClassifier(tf.keras.Model):
    def __init__(self, depth, start_f, num_classes, dense_units=512, activation_dense="relu",
                number_dense_layer=1, dropout=True):
        super(FirstCNNClassifier, self).__init__()
        
        self.feature_extractor = tf.keras.Sequential()
    
        for i in range(depth):
            self.feature_extractor.add(ConvBlock(num_filters=start_f))
            start_f *= 2
            
        self.flatten = tf.keras.layers.Flatten()
        self.classifier = tf.keras.Sequential()
        
        for i in range(number_dense_layer):
            self.classifier.add(tf.keras.layers.Dense(units=dense_units, activation=activation_dense))
        
        if dropout:
            self.classifier.add(tf.keras.layers.Dropout(rate=0.3, seed=SEED))
        
        self.classifier.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))
        
    def call(self, inputs):
        x = self.feature_extractor(inputs)
        x = self.flatten(x)
        x = self.classifier(x)
        return x

In [ ]:
# Create Model
# ------------

depth = 7
start_f = 7
num_classes = 20
    
# Create Model instance
model = FirstCNNClassifier(depth=depth,
                      start_f=start_f,
                      dense_units=1024,
                      num_classes=num_classes,
                      number_dense_layer=3,
                      dropout=True)
# Build Model (Required)
model.build(input_shape=(None, img_h, img_w, 3))

In [ ]:
model.feature_extractor.summary()

In [ ]:
model.summary()

In [ ]:
loss = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adadelta()
metrics = ['accuracy']
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [ ]:
train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))

valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen, 
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))
train_dataset = train_dataset.repeat()
valid_dataset = valid_dataset.repeat()

In [ ]:
import os
from datetime import datetime

# from tensorflow.compat.v1 import ConfigProto
# from tensorflow.compat.v1 import InteractiveSession

# config = ConfigProto()
# config.gpu_options.allow_growth = True
# session = InteractiveSession(config=config)

cwd = os.getcwd()

exps_dir = os.path.join(cwd, 'classification_experiments')
if not os.path.exists(exps_dir):
    os.makedirs(exps_dir)

now = datetime.now().strftime('%b%d_%H-%M-%S')

model_name = 'CNN_SECOND_MODEL'

exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)
    
callbacks = []

# Model checkpoint
# ----------------
ckpt_dir = os.path.join(exp_dir, 'ckpts')
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp_{epoch:02d}.ckpt'), 
                                                   save_weights_only=True)  # False to save the model directly
callbacks.append(ckpt_callback)

# Visualize Learning on Tensorboard
# ---------------------------------
tb_dir = os.path.join(exp_dir, 'tb_logs')
if not os.path.exists(tb_dir):
    os.makedirs(tb_dir)
    
# By default shows losses and metrics for both training and validation
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir,
                                             profile_batch=0,
                                             histogram_freq=1)  # if 1 shows weights histograms
callbacks.append(tb_callback)

# Early Stopping
# --------------
early_stop = True
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
    callbacks.append(es_callback)


model.fit(x=train_dataset,
          epochs=100,  #### set repeat in training dataset
          steps_per_epoch=len(train_gen),
          validation_data=valid_dataset,
          validation_steps=len(valid_gen), 
          callbacks=callbacks)

# How to visualize Tensorboard

# 1. tensorboard --logdir EXPERIMENTS_DIR --port PORT     <- from terminal
# 2. localhost:PORT   <- in your browser

In [ ]:
# Let's test data augmentation
# ----------------------------
import time
import matplotlib.pyplot as plt

%matplotlib notebook

fig = plt.figure()
ax = fig.gca()
fig.show()

iterator = iter(train_dataset)

for _ in range(1):
    augmented_img, target = next(iterator)
    augmented_img = augmented_img[0]   # First element
    augmented_img = augmented_img * 255  # denormalize
    
    plt.imshow(np.uint8(augmented_img))
    fig.canvas.draw()
    time.sleep(1)
    
print(target[0])

# THIRD MODEL

In [ ]:
import os
import tensorflow as tf
from datetime import datetime

if __name__ == '__main__':
    SEED = 482910
    tf.random.set_seed(SEED)

    cwd = os.getcwd()

    classes = [
        'owl',  # 1
        'galaxy',  # 2
        'lightning',  # 3
        'wine-bottle',  # ...
        't-shirt',
        'waterfall',
        'sword',
        'school-bus',
        'calculator',
        'sheet-music',
        'airplanes',
        'lightbulb',
        'skyscraper',
        'mountain-bike',
        'fireworks',
        'computer-monitor',
        'bear',
        'grand-piano',
        'kangaroo',
        'laptop'
    ]

    img_w = 400
    img_h = 400

    batch_size = 32
    num_classes = 20

    # ImageDataGenerator
    from tensorflow.keras.preprocessing.image import ImageDataGenerator

    train_data_gen = ImageDataGenerator(rotation_range=20,
                                        width_shift_range=1./20,
                                        height_shift_range=1./20,
                                        zoom_range=0.3,
                                        horizontal_flip=True,
                                        vertical_flip=True,
                                        fill_mode='constant',
                                        cval=0,
                                        rescale=1. / 255)

    valid_data_gen = ImageDataGenerator(rescale=1. / 255,
                                        fill_mode='constant',
                                        cval=0)



    data_dir = os.path.join(cwd, "..", "data", "Classification_Dataset")

    # Generator creation
    training_dir = os.path.join(data_dir, "training")
    valid_dir = os.path.join(data_dir, "validation")

    train_gen = train_data_gen.flow_from_directory(directory=training_dir,
                                                   target_size=(img_w, img_h),
                                                   classes=classes,
                                                   batch_size=batch_size,
                                                   shuffle=True,
                                                   seed=SEED)

    valid_gen = valid_data_gen.flow_from_directory(directory=valid_dir,
                                                   target_size=(img_w, img_h),
                                                   classes=classes,
                                                   batch_size=batch_size,
                                                   shuffle=False,
                                                   seed=SEED)

    # Dataset creation
    train_dataset = tf.data.Dataset.from_generator(lambda: train_gen, output_types=(tf.float32, tf.float32),
                                                   output_shapes=([None, img_h, img_w, 3], [None, num_classes]))
    train_dataset.repeat()

    valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen, output_types=(tf.float32, tf.float32),
                                                   output_shapes=([None, img_h, img_w, 3], [None, num_classes]))
    valid_dataset.repeat()

    ########## Model creation ##########
    model = tf.keras.models.Sequential()

    model.add(tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), padding='same', input_shape=(img_w, img_h, 3),
                                     activation='relu'))
    model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))

    for i in range(8 - 1):
        model.add(tf.keras.layers.Conv2D(filters=32 * (i + 1), kernel_size=(3, 3), padding='same', activation='relu'))
        model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))

    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(units=256, activation='relu'))
    model.add(tf.keras.layers.Dropout(rate=0.05, seed=SEED))
    model.add(tf.keras.layers.Dense(units=128, activation='relu'))
    model.add(tf.keras.layers.Dropout(rate=0.05, seed=SEED))
    model.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))

    model.build(input_shape=(batch_size, img_w, img_h, 3))
    model.summary()

    # Model compile
    optimizer = tf.keras.optimizers.Adam()
    model.compile(optimizer=optimizer, loss=tf.keras.losses.CategoricalCrossentropy(), metrics=["accuracy"])

    ########## Callbacks ##########
    exps_dir = os.path.join(cwd, 'classification_experiments')
    if not os.path.exists(exps_dir):
        os.makedirs(exps_dir)

    now = datetime.now().strftime('%b%d_%H-%M-%S')

    model_name = 'CNN end-to-end training'

    exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
    if not os.path.exists(exp_dir):
        os.makedirs(exp_dir)

    callbacks = []

    # Model checkpoint
    ckpt_dir = os.path.join(exp_dir, 'ckpts')
    if not os.path.exists(ckpt_dir):
        os.makedirs(ckpt_dir)

    ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp_{epoch:02d}.ckpt'),
                                                       save_weights_only=True)  # False to save the model directly
    callbacks.append(ckpt_callback)

    # Visualize Learning on Tensorboard
    tb_dir = os.path.join(exp_dir, 'tb_logs')
    if not os.path.exists(tb_dir):
        os.makedirs(tb_dir)

    tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir,
                                                 profile_batch=0,
                                                 histogram_freq=1)  # if 1 shows weights histograms
    callbacks.append(tb_callback)

    ########## Fitting ##########
    model.fit(x=train_dataset,
              epochs=70,
              steps_per_epoch=len(train_gen),
              validation_data=valid_dataset,
              validation_steps=len(valid_gen),
              callbacks=callbacks)



# Rescaled images...

In [ ]:
from src.data_management.data_reader import *

In [ ]:
root_path = os.getcwd()
root_path = os.path.join(root_path, "..")
train_set, valid_set, train_gen, valid_gen = read_training_data_unscaled(root_path)

In [ ]:
# Let's test data augmentation
# ----------------------------
import time
import matplotlib.pyplot as plt

%matplotlib notebook

fig = plt.figure()
ax = fig.gca()
fig.show()

iterator = iter(train_set)

for _ in range(10):
    augmented_img, target = next(iterator)
    augmented_img = augmented_img[0]   # First element
    
    plt.imshow(np.uint8(augmented_img))
    fig.canvas.draw()
    time.sleep(1)
    
print(target[0])